## Carga modulo comun

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../../common'))

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from purrfect.dataset import EPKADataset, load_partition,save_partition, create_train_valid_loaders
from torch.utils.data import DataLoader

from purrfect.training import train_model,train_validate
import torch.optim as optim

from purrfect.metrics import MetricAccumulator
from purrfect.active_learning import create_new_partition,create_next_partitions, test_model

from sklearn.model_selection import train_test_split

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Definición modelo

In [4]:
# Define the double convolution block
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

# Define the U-Net model
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        # Encoder
        self.encoder1 = DoubleConv(in_channels, 64)
        self.encoder2 = DoubleConv(64, 128)
        self.encoder3 = DoubleConv(128, 256)
        self.encoder4 = DoubleConv(256, 512)

        # Bottleneck
        self.bottleneck = DoubleConv(512, 1024)

        # Decoder
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = DoubleConv(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = DoubleConv(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = DoubleConv(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = DoubleConv(128, 64)

        # Final output layer
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        # Encoder
        e1 = self.encoder1(x)
        e2 = self.encoder2(F.max_pool2d(e1, 2))
        e3 = self.encoder3(F.max_pool2d(e2, 2))
        e4 = self.encoder4(F.max_pool2d(e3, 2))

        # Bottleneck
        b = self.bottleneck(F.max_pool2d(e4, 2))

        # Decoder with skip connections
        d4 = self.upconv4(b)
        d4 = torch.cat((e4, d4), dim=1)
        d4 = self.decoder4(d4)

        d3 = self.upconv3(d4)
        d3 = torch.cat((e3, d3), dim=1)
        d3 = self.decoder3(d3)

        d2 = self.upconv2(d3)
        d2 = torch.cat((e2, d2), dim=1)
        d2 = self.decoder2(d2)

        d1 = self.upconv1(d2)
        d1 = torch.cat((e1, d1), dim=1)
        d1 = self.decoder1(d1)

        # Final output
        out = self.final_conv(d1)
        return out

## Creación particion inicial

In [6]:
#Creacion de particiones train y valid
init_partition = load_partition("partition_1.json")
train_partition, val_partition = train_test_split(init_partition, test_size=0.2, random_state=42)
save_partition("partition_1_train.json","partitions",train_partition)
save_partition("partition_1_val.json","partitions",val_partition)

## Carga modelo

In [6]:
#Define model
model = UNet( 3, 1)
model = model.to(DEVICE)
# Define Loss
criterion = torch.nn.L1Loss()
current_partition = 1

In [8]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Train Epoch 1: Loss: 0.128194 WMAPE: 42.3407 DICE: 0.5471 DPEAKS: 223.5542


Validate Epoch 1: Loss: 0.039698 WMAPE: 10.0609 DICE: 0.5458 DPEAKS: 215.9657
Saving best model
Epoch [2/50]


Train Epoch 2: Loss: 0.032825 WMAPE: 7.0331 DICE: 0.5445 DPEAKS: 212.8914


Validate Epoch 2: Loss: 0.028768 WMAPE: 5.2219 DICE: 0.5392 DPEAKS: 214.8694
Saving best model
Epoch [3/50]


...

Validate Epoch 15: Loss: 0.018467 WMAPE: 5.4358 DICE: 0.3016 DPEAKS: 137.9853
Epoch [16/50]


Train Epoch 16: Loss: 0.012710 WMAPE: 3.6601 DICE: 0.2069 DPEAKS: 101.3999


Validate Epoch 16: Loss: 0.018237 WMAPE: 4.8067 DICE: 0.3006 DPEAKS: 140.0441
Epoch [17/50]


Train Epoch 17: Loss: 0.012410 WMAPE: 3.6012 DICE: 0.2011 DPEAKS: 97.6467


Validate Epoch 17: Loss: 0.018339 WMAPE: 5.2905 DICE: 0.3017 DPEAKS: 137.5682
early stopping: 3 epochs without improvement
Training complete.


In [9]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)

Validate Epoch test: Loss: 0.018219 WMAPE: 4.9172 DICE: 0.2995 DPEAKS: 136.4305


In [10]:
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_2_train: Loss: 0.018199 WMAPE: 4.8809 DICE: 0.3005 DPEAKS: 138.5840


Validate Epoch partition_2_val: Loss: 0.018183 WMAPE: 5.0124 DICE: 0.3010 DPEAKS: 140.9910


In [ ]:
model.load_state_dict(torch.load(best_model_path,weights_only=True))

In [10]:
current_partition+=1

In [11]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Train Epoch 1: 100%|██████████| 307/307 [02:08<00:00,  2.39it/s, loss=0.0176]


Train Epoch 1: Loss: 0.015529 WMAPE: 4.6847 DICE: 0.2651 DPEAKS: 136.9173


Validate Epoch 1: 100%|██████████| 77/77 [00:11<00:00,  6.63it/s, loss=0.0181]


Validate Epoch 1: Loss: 0.017552 WMAPE: 5.2040 DICE: 0.2944 DPEAKS: 146.7108
Saving best model
Epoch [2/50]


...

Validate Epoch 4: Loss: 0.017970 WMAPE: 5.0236 DICE: 0.2989 DPEAKS: 149.1332
Epoch [5/50]


Train Epoch 5: 100%|██████████| 307/307 [02:09<00:00,  2.37it/s, loss=0.0147]


Train Epoch 5: Loss: 0.012796 WMAPE: 4.0814 DICE: 0.2179 DPEAKS: 115.7515


Validate Epoch 5: 100%|██████████| 77/77 [00:11<00:00,  6.55it/s, loss=0.018] 


Validate Epoch 5: Loss: 0.017880 WMAPE: 5.5960 DICE: 0.3031 DPEAKS: 150.3480
early stopping: 3 epochs without improvement
Training complete.


In [13]:
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_3_train: 100%|██████████| 307/307 [00:50<00:00,  6.04it/s, loss=0.0179]


Validate Epoch partition_3_train: Loss: 0.017745 WMAPE: 4.5746 DICE: 0.2878 DPEAKS: 129.1435


Validate Epoch partition_3_val: 100%|██████████| 77/77 [00:12<00:00,  6.03it/s, loss=0.0198]

Validate Epoch partition_3_val: Loss: 0.017645 WMAPE: 4.5229 DICE: 0.2868 DPEAKS: 133.8457


In [9]:
current_partition=3
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)

In [9]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [3/50]


Validate Epoch 3: 100%|██████████| 77/77 [00:11<00:00,  6.64it/s, loss=0.0184]


Epoch [4/50]


Validate Epoch 4: 100%|██████████| 77/77 [00:11<00:00,  6.65it/s, loss=0.0174]


Epoch [5/50]


Validate Epoch 5: 100%|██████████| 77/77 [00:11<00:00,  6.63it/s, loss=0.0181]


early stopping: 3 epochs without improvement
Training complete.


In [10]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_4_val: 100%|██████████| 77/77 [00:12<00:00,  6.25it/s, loss=0.0168]


In [8]:
current_partition=4

In [9]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:11<00:00,  6.47it/s, loss=0.0462]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:11<00:00,  7.00it/s, loss=0.0335]


Saving best model
Epoch [3/50]


...

Epoch [16/50]


Validate Epoch 16: 100%|██████████| 77/77 [00:10<00:00,  7.06it/s, loss=0.018] 


Epoch [17/50]


Validate Epoch 17: 100%|██████████| 77/77 [00:10<00:00,  7.05it/s, loss=0.0181]


early stopping: 3 epochs without improvement
Training complete.


In [10]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_5_val: 100%|██████████| 77/77 [00:12<00:00,  6.09it/s, loss=0.0192]


In [7]:
current_partition=5

In [8]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [8/50]


Validate Epoch 8: 100%|██████████| 77/77 [00:11<00:00,  6.61it/s, WMAPE=5.91, DICE=0.289, DPEAKS=146, Loss=0.0178]


Epoch [9/50]


Validate Epoch 9: 100%|██████████| 77/77 [00:11<00:00,  6.56it/s, WMAPE=5.66, DICE=0.289, DPEAKS=150, Loss=0.0196]


early stopping: 3 epochs without improvement
Training complete.


In [9]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_6_val: 100%|██████████| 77/77 [00:12<00:00,  6.01it/s, WMAPE=4.67, DICE=0.272, DPEAKS=128, Loss=0.0176]
